In [5]:
import boto3
from botocore.client import Config
import os

class S3:
    def __init__(self):
        self.S3_ACCESS_KEY = os.environ.get("S3_ACCESS_KEY", "")
        self.S3_SECRET_KEY = os.environ.get("S3_SECRET_KEY", "")
        self.S3_ENDPOINT = os.environ.get("S3_ENDPOINT", "")
        self.S3_REGION = os.environ.get("S3_REGION", "")
        self.bucket_name = os.environ.get("S3_BUCKET", "")
        
        self.s3_client = boto3.client(
            "s3",
            aws_access_key_id=self.S3_ACCESS_KEY,
            aws_secret_access_key=self.S3_SECRET_KEY,
            endpoint_url=self.S3_ENDPOINT,
            region_name=self.S3_REGION
        )

    def list_buckets(self):
        response = self.s3_client.list_buckets()
        print("Buckets existants :")
        for bucket in response['Buckets']:
            print(f'  {bucket["Name"]}')

    def list_objects(self):
        response = self.s3_client.list_objects_v2(Bucket=self.bucket_name)
        print(f"Objets dans le bucket {self.bucket_name} :")
        for obj in response.get('Contents', []):
            print(f'  {obj["Key"]}')

    def upload_file(self, file_path, object_name):
        try:
            self.s3_client.upload_file(file_path, self.bucket_name, object_name, ExtraArgs={'ACL': 'public-read'})
            print(f"File {file_path} uploaded successfully to bucket {self.bucket_name} as {object_name}.")
            
            # Generate the public URL
            public_url = f"{self.S3_ENDPOINT}/{self.bucket_name}/{object_name}"
            print(f"Public URL: {public_url}")
            return public_url
        except Exception as e:
            print(f"Error during file upload: {e}")
            return None

    def get_presigned_url(self, object_name, expiration=3600):
        return self.s3_client.generate_presigned_url(
            ClientMethod='get_object',
            Params={'Bucket': self.bucket_name, 'Key': object_name},
            ExpiresIn=expiration  # URL expiration time in seconds
        )
    
    def remove_file(self, object_name):
        try:
            self.s3_client.delete_object(Bucket=self.bucket_name, Key=object_name)
            print(f"File {object_name} successfully removed from bucket {self.bucket_name}.")
            return True
        except Exception as e:
            print(f"Error during file removal: {e}")
            return False


In [6]:
import sys
import dotenv
dotenv.load_dotenv('../../.env')

s3 = S3()

s3.list_buckets()

s3.list_objects()

file_path = './documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4'
object_name = '/documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4'
public_url = s3.upload_file(file_path, object_name)

if public_url:
    presigned_url = s3.get_presigned_url(object_name)
    print(f"Presigned URL: {presigned_url}")


Buckets existants :
  collectif-objets-development
  dev-atnum-state-s3
  languia-01-dev-main
  pop-dev-assets
  pop-dev-backup
  pop-dev-general
  pop-dev-perf-assets
  pop-dev-sitemaps
  pop-os-backup
  pop-test-rclone
  science-infuse-content
Objets dans le bucket science-infuse-content :
File ./documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4 uploaded successfully to bucket science-infuse-content as /documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4.
Public URL: https://s3.gra.io.cloud.ovh.net//science-infuse-content//documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4
Presigned URL: https://s3.gra.io.cloud.ovh.net/science-infuse-content//documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=d23ef1dff2cb462bb1f44dfcd01fca6e%2F20240710%2Fgra%2Fs3%2Faws4_request&X-Amz-Date=20240710T095836Z&X-Amz-Expires=3600&X-Amz-SignedHeaders=host&X-Amz-Signature=7ba1be71dc995438cd42e1fbaa875553b354f086644af6e399d48d81

In [4]:
s3.list_objects()
s3.remove_file(object_name)
s3.list_objects()

Objets dans le bucket science-infuse-content :
  /documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4
File /documents/youtube/40696c1b-6561-49ca-bb49-d20990d87040.mp4 successfully removed from bucket science-infuse-content.
Objets dans le bucket science-infuse-content :
